In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os.path import join
import seaborn as sns
import time 

**Read the file with PANDAS**

In [10]:
start_time = time.process_time()
pd_data = pd.read_csv('yelp_academic_dataset_review.csv')
print(time.process_time() - start_time, "seconds")

77.49564344400001 seconds


In [11]:
pd_data.head(2)

,user_id,text,date,review_id,business_id,funny,cool,useful,stars
0,b'hG7b0MtEbXx5QzbzE6C_VA',b'Total bill for this horrible service? Over $...,b'2013-05-07 04:34:36',b'Q1sbwvVQXV2734tPgoKj4Q',b'ujmEBvifdJM6h6RLv4wQIg',1,0,6,1.0
1,b'yXQM5uF2jS6es16SJzNHfg',"b""I *adore* Travis at the Hard Rock's new Kell...",b'2017-01-14 21:30:33',b'GJXCdrto3ASJOqKeVWPi6Q',b'NZnhc2sEQy3RmzKTZnqtwQ',0,0,0,5.0


In [12]:
pd_data.shape


(6685900, 9)

In [13]:
%pip install "dask[complete]"

**Read the file with DASK**

In [14]:
!python -m pip install "dask[dataframe]"

In [15]:
import dask.dataframe 

In [16]:
#path = "/content/drive/MyDrive/Colab Notebooks/yelp_academic_dataset_review.csv.zip"
import time
start_time = time.process_time()
dask_data = dask.dataframe.read_csv('yelp_academic_dataset_review.csv')
print(time.process_time() - start_time, "seconds")


0.02197610100000702 seconds


Since the process time of dask is much less than pandas we will continue on preprocessing step with dask

In [18]:
dask_data.drop_duplicates(subset="user_id", keep='first', inplace=True)
dask_data.head()

,user_id,text,date,review_id,business_id,funny,cool,useful,stars
0,b'hG7b0MtEbXx5QzbzE6C_VA',b'Total bill for this horrible service? Over $...,b'2013-05-07 04:34:36',b'Q1sbwvVQXV2734tPgoKj4Q',b'ujmEBvifdJM6h6RLv4wQIg',1,0,6,1.0
1,b'yXQM5uF2jS6es16SJzNHfg',"b""I *adore* Travis at the Hard Rock's new Kell...",b'2017-01-14 21:30:33',b'GJXCdrto3ASJOqKeVWPi6Q',b'NZnhc2sEQy3RmzKTZnqtwQ',0,0,0,5.0
2,b'n6-Gk65cPZL6Uz8qRm3NYw',"b""I have to say that this office really has it...",b'2016-11-09 20:09:03',b'2TzJjDVDEuAW6MR5Vuc1ug',b'WTqjgwHlXbSFevF32_DJVw',0,0,3,5.0
3,b'dacAIZ6fTM6mqwW5uxkskg',"b""Went in for a lunch. Steak sandwich was deli...",b'2018-01-09 20:56:38',b'yi0R0Ugj_xUx_Nek0-_Qig',b'ikCg8xy5JIg_NGPx-MSIDA',0,0,0,5.0
4,b'ssoyf2_x0EQMed6fgHeMyQ',b'Today was my second out of three sessions I ...,b'2018-01-30 23:07:38',b'11a8sVPMUFtaC7_ABRkmtw',b'b1b1eb3uo-w561D0ZfCEiQ',0,0,7,1.0


In [19]:
dask_data.isnull().sum().sum()

dd.Scalar<series-..., dtype=int64>

**Read the file with MODIN**

In [22]:
pip install modin[ray]

In [ ]:
import modin.pandas as pdd
import time
#path = "/content/drive/MyDrive/Colab Notebooks/yelp_academic_dataset_review.csv.zip"
start_time = time.process_time()
modin_data = pdd.read_csv('yelp_academic_dataset_review.csv')
print(time.process_time() - start_time, "seconds")


To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()

tcmalloc: large alloc 2426716160 bytes == 0x563d9b0d8000 @  0x7f08c76491e7 0x563c664f6a18 0x563c664c1987 0x563c665a36be 0x563c664c4ea9 0x563c665b6c0d 0x563c665390d8 0x563c66533c35 0x563c664c673a 0x563c6653593b 0x563c66533c35 0x563c664c673a 0x563c6653593b 0x563c66533c35 0x563c664c6fec 0x563c664c71f1 0x563c66536318 0x563c66534235 0x563c664c6fec 0x563c664c71f1 0x563c66536318 0x563c66533c35 0x563c664c6fec 0x563c664c71f1 0x563c66536318 0x563c66533c35 0x563c664c6fec 0x563c664c71f1 0x563c66536318 0x563c66534235 0x563c66405e2c
tcmalloc: large alloc 2426716160 bytes == 0x563d9b0d8000 @  0x7f08c76491e7 0x563c664f6a18 0x563c664c1987 0x563c665a36be 0x563c664c4ea9 0x563c665b6c0d 0x563c665390d8 0x563c66533c35 0x563c664c673a 0x563c6653593b 0x563c66533c35 0x563c664c673a 0x563c6653593b 0x563c66533c35 0x563c664c6fec 0x563c664c71f1 0x563c66536318 0x563c66534235 0x563c664c6fec 0x563c664

**YAML file creation**

In [1]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re


################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Overwriting testutility.py


In [2]:
%%writefile yaml_file.yaml
#WRITE YAML FILE
file_type: csv
file_name: yelp_academic_dataset_review
dataset_name: df
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns:  
    - user_id
    - text
    - date
    - review_id
    - business_id
    - funny
    - cool
    - useful
    - stars

Overwriting yaml_file.yaml


In [3]:
import testutility as util
config_data = util.read_config_file("yaml_file.yaml")

In [4]:
config_data

{'file_type': 'csv',
 'file_name': 'yelp_academic_dataset_review',
 'dataset_name': 'df',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['user_id',
  'text',
  'date',
  'review_id',
  'business_id',
  'funny',
  'cool',
  'useful',
  'stars']}

In [6]:
config_data['inbound_delimiter']

','

In [9]:
import pandas as pd
file_type = config_data['file_type']
source_file = "./" + config_data['file_name'] + f'.{file_type}'
#print("",source_file)
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3441: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


,user_id,text,date,review_id,business_id,funny,cool,useful,stars
0,b'hG7b0MtEbXx5QzbzE6C_VA',b'Total bill for this horrible service? Over $...,b'2013-05-07 04:34:36',b'Q1sbwvVQXV2734tPgoKj4Q',b'ujmEBvifdJM6h6RLv4wQIg',1,0,6,1.0
1,b'yXQM5uF2jS6es16SJzNHfg',"b""I *adore* Travis at the Hard Rock's new Kell...",b'2017-01-14 21:30:33',b'GJXCdrto3ASJOqKeVWPi6Q',b'NZnhc2sEQy3RmzKTZnqtwQ',0,0,0,5.0
2,b'n6-Gk65cPZL6Uz8qRm3NYw',"b""I have to say that this office really has it...",b'2016-11-09 20:09:03',b'2TzJjDVDEuAW6MR5Vuc1ug',b'WTqjgwHlXbSFevF32_DJVw',0,0,3,5.0
3,b'dacAIZ6fTM6mqwW5uxkskg',"b""Went in for a lunch. Steak sandwich was deli...",b'2018-01-09 20:56:38',b'yi0R0Ugj_xUx_Nek0-_Qig',b'ikCg8xy5JIg_NGPx-MSIDA',0,0,0,5.0
4,b'ssoyf2_x0EQMed6fgHeMyQ',b'Today was my second out of three sessions I ...,b'2018-01-30 23:07:38',b'11a8sVPMUFtaC7_ABRkmtw',b'b1b1eb3uo-w561D0ZfCEiQ',0,0,7,1.0


In [10]:
util.col_header_val(df,config_data)

column name and column length validation passed


1

In [11]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['user_id', 'text', 'date', 'review_id', 'business_id', 'funny', 'cool',
       'useful', 'stars'],
      dtype='object')
columns of YAML are: ['user_id', 'text', 'date', 'review_id', 'business_id', 'funny', 'cool', 'useful', 'stars']


In [12]:
df.to_csv('df_gz.csv.gz', compression='gzip')

In [2]:
import pandas as pd
df4 = pd.read_csv("df_gz.csv.gz")

In [3]:
df4.head(5)

,Unnamed: 0,user_id,text,date,review_id,business_id,funny,cool,useful,stars
0,0,b'hG7b0MtEbXx5QzbzE6C_VA',b'Total bill for this horrible service? Over $...,b'2013-05-07 04:34:36',b'Q1sbwvVQXV2734tPgoKj4Q',b'ujmEBvifdJM6h6RLv4wQIg',1,0,6,1.0
1,1,b'yXQM5uF2jS6es16SJzNHfg',"b""I *adore* Travis at the Hard Rock's new Kell...",b'2017-01-14 21:30:33',b'GJXCdrto3ASJOqKeVWPi6Q',b'NZnhc2sEQy3RmzKTZnqtwQ',0,0,0,5.0
2,2,b'n6-Gk65cPZL6Uz8qRm3NYw',"b""I have to say that this office really has it...",b'2016-11-09 20:09:03',b'2TzJjDVDEuAW6MR5Vuc1ug',b'WTqjgwHlXbSFevF32_DJVw',0,0,3,5.0
3,3,b'dacAIZ6fTM6mqwW5uxkskg',"b""Went in for a lunch. Steak sandwich was deli...",b'2018-01-09 20:56:38',b'yi0R0Ugj_xUx_Nek0-_Qig',b'ikCg8xy5JIg_NGPx-MSIDA',0,0,0,5.0
4,4,b'ssoyf2_x0EQMed6fgHeMyQ',b'Today was my second out of three sessions I ...,b'2018-01-30 23:07:38',b'11a8sVPMUFtaC7_ABRkmtw',b'b1b1eb3uo-w561D0ZfCEiQ',0,0,7,1.0


In [4]:
df4.shape

(6685900, 10)

In [5]:
import os

#FUNCTION THAT CALCULATE TOTAL SIZE IN BYTES OF THE COMPRESSED FOLDER
def get_size(start_path = '.'):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(start_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            # skip if it is symbolic link
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)

    return total_size

In [6]:

from pathlib import Path
Path('yelp_academic_dataset_review.csv').stat().st_size

4853420661

In [8]:
from pathlib import Path
Path('yelp_academic_dataset_review.csv').stat().st_size

4853420661